# Quick Start Guide

## Introduction

This quick start guide aims to briefly introduce you to GAMSPy and its main features. After completing this guide you will be able to implement basic mathematical models using GAMSPy. For advanced features we advise you to look into our [Users Guide](https://gamspy.readthedocs.io/en/latest/user/index.html) and the [model library](https://gamspy.readthedocs.io/en/latest/user/model_library.html).

To successfully complete this tutorial it is helpful, but not mandatory, to have basic knowledge of programming in Python and the use of the [Pandas library](https://pandas.pydata.org/).

## A Transportation Problem

The example we will use in this guide is an instance of the transportation problem. We are given the supplies at several plants and the demands at several markets for a single commodity, and we are given the unit costs of shipping the commodity from plants to markets. The economic question is: how much shipment should there be between each plant and each market so as to minimize total transport cost?

The algebraic representation of this problem is usually presented in a format similar to the following.

Indices:

- $i$ = plants
- $j$ = markets

Given Data:

- $a_i$ = supply of commodity of plant $i$ (in cases)
- $b_j$ = demand for commodity at market $j$
- $c_{ij}$ = cost per unit shipment between plant $i$ and market $j$

Decision Variables:

- $x_{ij}$ = amount of commodity to ship from plant $i$ to market $j$ where $x_{ij} \ge 0$ for all $i,j$

Constraints:

- Observe supply limit at plant $i$: $\sum_j x_{ij} \le a_i \: \forall i$
- Satisfy demand at market $j$: $\sum_i x_{ij} \ge b_j \: \forall j$
- Objective Function: Minimize $\sum_i \sum_j c_{ij} \cdot x_{ij}$


## Data

We start by using the Pandas library to include the data which we will later pass to our optimization model.


In [78]:
import pandas as pd

capacities = pd.DataFrame(
    [["seattle", 350], ["san-diego", 600]], columns=["city", "capacity"]
).set_index("city")

capacities

,capacity
city,
seattle,350
san-diego,600


In [79]:
demands = pd.DataFrame(
    [["new-york", 325], ["chicago", 300], ["topeka", 275]], columns=["city", "demand"]
).set_index("city")

demands

,demand
city,
new-york,325
chicago,300
topeka,275


In [80]:
distances = pd.DataFrame(
    [
        ["seattle", "new-york", 2.5],
        ["seattle", "chicago", 1.7],
        ["seattle", "topeka", 1.8],
        ["san-diego", "new-york", 2.5],
        ["san-diego", "chicago", 1.8],
        ["san-diego", "topeka", 1.4],
    ],
    columns=["from", "to", "distance"],
).set_index(["from", "to"])

distances

distance
from      to                
seattle   new-york       2.5
          chicago        1.7
          topeka         1.8
san-diego new-york       2.5
          chicago        1.8
          topeka         1.4

In [81]:
freight_cost = 90

## Symbol Declaration

Similar to our separation of the transportation problem into sets, parameter, variables, and constraints we will use this separation to define the transportation problem as a GAMSPy model. Therefore, we need to import the `gamspy` library first.


In [82]:
from gamspy import Model, Container, Sum, Sense

### Container

A container gathers all the information that is relevant for a GAMSPy model. Thus, the first step is to create such a container.


In [83]:
m = Container()

### Sets

Sets are the basic building blocks of a GAMSPy model, corresponding exactly to the indices in the algebraic representations of models.

Indices of the transportation problem:

- $i$ = plants
- $j$ = markets

Sets are associated with a container, which is why we use `Container().addSet()`. You can find further information on the usage of sets in the [user guide](https://gamspy.readthedocs.io/en/latest/user/set.html).

There a two ways to declare sets:

1. Separate declaration and data assignment
2. Combine declaration and data assignment

#### Separate declaration and data assignment


In [84]:
i = m.addSet(name="i", description="plants")
i.setRecords(capacities.index)

#### Combine declaration and data assignment


In [85]:
j = m.addSet(name="j", description="markets", records=demands.index)

The effect of using the above `m.addSet()` statements is that we declared two sets and gave them the names i and j. We added a description to elaborate on the meaning and make the model easier readable. Lastly, we assigned members to the sets as follows:

$i$ = {Seattle, San Diego}

$j$ = {New York, Chicago, Topeka}

To verify, you can use `<python variable>.records`.


In [86]:
i.records

,uni,element_text
0,seattle,
1,san-diego,


In [87]:
j.records

,uni,element_text
0,new-york,
1,chicago,
2,topeka,


### Parameters

To declare parameters we use `<container>.addParameter()`. We give the parameter a name and a description. Note that parameter $a_i$ is indexed by $i$. To account for indices we add the `domain` attribute and point to the corresponding set.

Note that you can either, as shown for sets, combine or separate the declaration and the data assignment. For convenience we combine declaration and data assignment from now on.


In [88]:
a = m.addParameter(
    name="a",
    domain=i,
    description="supply of commodity of plant i (in cases)",
    records=capacities.reset_index(),
)
a.records

,city,value
0,seattle,350.0
1,san-diego,600.0


In [89]:
b = m.addParameter(
    name="b",
    domain=j,
    description="demand for commodity at market j",
    records=demands.reset_index(),
)
b.records

,city,value
0,new-york,325.0
1,chicago,300.0
2,topeka,275.0


In [90]:
c = m.addParameter(
    name="c",
    domain=[i, j],
    description="cost per unit shipment between plant i and market j",
)

The cost per unit shipment between plant $i$ and market $j$ is derived from the distance between $i$ and $j$ and can be calculated as follows:

$c_{ij} = \frac{90 \cdot d_{ij}}{1000}$,

where $d_{ij}$ denotes the distance between $i$ and $j$.

We have two options to calculate $c_{ij}$ and assign the data to the GAMSPy parameter:

1. Python assignment - calculation in Python, e.g., using Pandas and `.setRecords()`
2. GAMSPy assignment - calculation on GAMSPy

#### Python Assignment


In [91]:
cost = freight_cost * distances / 1000
cost

distance
from      to                
seattle   new-york     0.225
          chicago      0.153
          topeka       0.162
san-diego new-york     0.225
          chicago      0.162
          topeka       0.126

In [92]:
c.setRecords(cost.reset_index())
c.records

,from,to,value
0,seattle,new-york,0.225
1,seattle,chicago,0.153
2,seattle,topeka,0.162
3,san-diego,new-york,0.225
4,san-diego,chicago,0.162
5,san-diego,topeka,0.126


#### GAMSPy Assignment

For the direct assignment we need to declare a new parameter denoting the distances between $i$ and $j$.


In [93]:
d = m.addParameter(
    name="d",
    domain=[i, j],
    description="distance between plant i and market j",
    records=distances.reset_index(),
)
d.records

,from,to,value
0,seattle,new-york,2.5
1,seattle,chicago,1.7
2,seattle,topeka,1.8
3,san-diego,new-york,2.5
4,san-diego,chicago,1.8
5,san-diego,topeka,1.4


In [94]:
c[i, j] = freight_cost * d[i, j] / 1000
c.records

,i,j,value
0,seattle,new-york,0.225
1,seattle,chicago,0.153
2,seattle,topeka,0.162
3,san-diego,new-york,0.225
4,san-diego,chicago,0.162
5,san-diego,topeka,0.126


### Variables

The decision variables of a GAMSPy-expressed model must be declared with the `container.addVariable()` statement. Each variable is given a name, a domain if appropriate, a type, and (optionally) description.


In [95]:
x = m.addVariable(
    name="x",
    domain=[i, j],
    type="Positive",
    description="amount of commodity to ship from plant i to market j",
)

This statement results in the declaration of a shipment variable for each (i,j) pair. (You will see in chapter [Equations](#equations), how GAMSPy can handle the typical real-world situation in which only a subset of the (i,j) pairs is allowable for shipment.)

More information on variables can be found in the [variable section](https://gamspy.readthedocs.io/en/latest/user/variable.html) of our user guide.


## Equations
Equations must be declared and defined in separate statements. The format of the declaration is the same as for other GAMSPy entities. First comes the keyword, `m.addEquation()` in this case, followed by the name, domain and text. The transportation problem has two constraints:

Supply: observe supply limit at plant $i$: $\sum_j x_{ij} \le a_i \: \forall i$
Demand: satisfy demand at market $j$: $\sum_i x_{ij} \ge b_j \: \forall j$

In [96]:
supply = m.addEquation(
    name="supply", domain=i, description="observe supply limit at plant i"
)
demand = m.addEquation(
    name="demand", domain=j, description="satisfy demand at market j"
)

The components of an equation definition are:
1. The Python variable of the equation being defined
2. The domain (optional)
3. Domain restriction condition (optional)
4. A `=` sign
5. Left hand side expression
6. Relational operator (`==`, `<=`, `>=`)
7. The right hand side expression.

The equation definition for the supply constraint of the transportation problem is implemented as follows:

In [97]:
supply[i] = Sum(j, x[i, j]) <= a[i]

Using the same logic as above, we can define the demand equation as follows:

In [98]:
demand[j] = Sum(i, x[i, j]) >= b[j]

More information on equations is given in the [equation section](https://gamspy.readthedocs.io/en/latest/user/equation.html) of our user guide.

## Objective
The objective function of a GAMSPy model does not require a separate equation declaration. You can assign the objective expression to a Python variable or use it directly in the model statement of the [next section](#model).


In [99]:
obj = Sum((i, j), c[i, j] * x[i, j])

## Model
A GAMSPy `Model()` is a collection of the constraints that go into your model. It is also associated with an objective function, a sense (minimize or maximize), and a problem type. Further, a GAMSPy `Model()` has a name and is associated with a `Container()`.

To define the transportation problem as a GAMSPy `Model()`, we assign the `Model()` to a Python variable, point to our container `m` that we filled with GAMSPy symbols and data, and give it the name `transport`. Further, we define th problem type to be linear (LP), specify the sense to minimization (`Sense.MIN`), and point to the objective expression.

GAMSPy allows to alternatives to assign equations to a `Model()`:
1. Using a list of equations,
2. Retrieving _all_ equations by calling `container.getEquations()`.

### Using a List of Equations
Using a list of equations is especially useful if you want to define multiple GAMSPy models with a subset of all equations in your container. For the transportation problem this can be done as follows:

In [100]:
transport = Model(
    m,
    name="transport",
    equations=[supply, demand],
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

### Retrieving all Equations
Using `Container().getEquations()` is especially convenient if you want to include all equations of your container to be associated with your model. For the transportation problem this can be done as follows:

In [101]:
transport_2 = Model(
    m,
    name="transport",
    equations=m.getEquations(),
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

More information on the usage of a GAMSPy `Model()` can be found in the [model section](https://gamspy.readthedocs.io/en/latest/user/model.html) of our user guide.

## Solve
After you have defined you GAMSPy `Model()`, you are ready to solve it.  The solve statement will cause several things to happen when executed. The specific instance of interest of the model will be generated, the appropriate data structures for inputting this problem to the solver will be created, and the solver will be invoked. If you want to see the output from the solver to be written to your console, you can use the `sys` libaray and pass the `output=sys.stdout` attribute to the `Model().solve()` statement.

In [102]:
import sys

transport.solve(output=sys.stdout)

--- Job _gams_py_gjo5.gms Start 10/06/23 11:24:25 45.0.0 6d29854d WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\jbroi\anaconda3\envs\test\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\jbroi\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Restart C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\default_save.g00
    Input C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\_gams_py_gjo5.gms
    Output C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\_gams_py_gjo5.lst
    Save C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\default_restart.g00
    ScrDir C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\225a\
    SysDir C:\Users\jbroi\anaconda3\envs\test\Lib\site-packages\gamspy_base\
    CurDir C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\
    L

GamsExceptionExecution: GAMS return code not 0 (2), check C:\Users\jbroi\AppData\Local\Temp\tmpb57wt3c9\_gams_py_gjo5.lst for more details

# Results


In [ ]:
x_s = x.records.set_index(["i", "j"])

x_s

level  marginal  lower  upper  scale
i         j                                             
seattle   new-york    0.0    -0.000    0.0    inf    1.0
          chicago   300.0     0.000    0.0    inf    1.0
          topeka      0.0     0.036    0.0    inf    1.0
san-diego new-york  325.0     0.000    0.0    inf    1.0
          chicago     0.0     0.009    0.0    inf    1.0
          topeka    275.0     0.000    0.0    inf    1.0

In [ ]:
transport.objective_value

153.675

In [ ]:
transport.status

<ModelStatus.OptimalGlobal: 1>